In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555148 sha256=0bf3aa0096be1eaeb0ea274e736c28ef09159cc3545f816478d889377dcf51c4
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 re

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import GridSearchCV, cross_validate


In [3]:
ratings = pd.read_csv("ml-32m/ratings.csv")

In [4]:
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [5]:
ratings['movieId'] = ratings['movieId'].astype(int)

In [6]:
print(ratings.dtypes)

userId         int64
movieId        int64
rating       float64
timestamp    float64
dtype: object


Підбір найкращих параметрів для SVD

In [7]:
param_grid = {
    'n_factors': [20, 50],
    'reg_all': [0.02, 0.1],
    'lr_all': [0.002]
}

gs = GridSearchCV(
    SVD,
    param_grid,
    measures=['rmse', 'mae'],
    cv=3,
    n_jobs=-1
)

print("Пошук найкращих параметрів SVD..")
gs.fit(data)

print("\nНайкращі параметри для SVD (за RMSE):")
print(gs.best_params['rmse'])
print("Найкращий RMSE:", gs.best_score['rmse'])

Пошук найкращих параметрів SVD..

Найкращі параметри для SVD (за RMSE):
{'n_factors': 20, 'reg_all': 0.02, 'lr_all': 0.002}
Найкращий RMSE: 0.8753756223227974


Оцінка моделей: SVD, SVD++, NMF

In [8]:
algorithms = {
    "SVD (best params)": SVD(**gs.best_params['rmse']),
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results = {}

print("\nКрос-валідація моделей..")
for name, algo in algorithms.items():
    cv_results = cross_validate(
        algo,
        data,
        measures=['RMSE', 'MAE'],
        cv=3,
        n_jobs=-1,
        verbose=False
    )

    results[name] = {
        'rmse': cv_results['test_rmse'].mean(),
        'mae': cv_results['test_mae'].mean()
    }


Крос-валідація моделей..


Результати:

In [11]:
print("\nПорівнюємо моделі:")
for model, metrics in results.items():
    print(f"{model}: RMSE={metrics['rmse']:.4f}, MAE={metrics['mae']:.4f}")

best_model = min(results, key=lambda x: results[x]['rmse'])
print("\nНайкраща модель(за RMSE):", best_model)


Порівнюємо моделі:
SVD (best params): RMSE=0.8755, MAE=0.6671
SVD++: RMSE=0.8395, MAE=0.6336
NMF: RMSE=0.8980, MAE=0.6808

Найкраща модель(за RMSE): SVD++
